知识点：
    
    1.基础知识在笔记本，E盘的数据挖掘课程。学习了aproiri和FP growth算法
    2.Python中的实现：
        （1）手动根据写代码
        （2）Python中的包：mlxtend库、pymining、Orange3库，https://jingyan.baidu.com/article/925f8cb8b564e7c0dde056cc.html
            https://www.jianshu.com/p/dcee9037a88f
        

In [8]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'E:\BaiduDownload\Learning-Data-Mining-with-Python-master\Chapter 4\电影推荐-亲和性分析\ratings.csv',engine='python')
df['timestamp']=pd.to_datetime(df['timestamp'],unit='s')
df.columns = ['UserID','MovieID','Rating','Datetime']
df.head()

,UserID,MovieID,Rating,Datetime
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51


In [26]:
#先确定用户是不是喜欢某部电影
df=df.drop('Favorible',axis = 1)
df['Favorable'] = df['Rating']>3
df.head()

,UserID,MovieID,Rating,Datetime,Favorable
0,1,1,4.0,2000-07-30 18:45:03,True
1,1,3,4.0,2000-07-30 18:20:47,True
2,1,6,4.0,2000-07-30 18:37:04,True
3,1,47,5.0,2000-07-30 19:03:35,True
4,1,50,5.0,2000-07-30 18:48:51,True


In [28]:
ratings = df[df['UserID'].isin(range(200))]
favorable_ratings = ratings[ratings['Favorable']]
print(ratings.head(),ratings.shape)
print(favorable_ratings.head(),favorable_ratings.shape)
print(df.shape)

   UserID  MovieID  Rating            Datetime  Favorable
0       1        1     4.0 2000-07-30 18:45:03       True
1       1        3     4.0 2000-07-30 18:20:47       True
2       1        6     4.0 2000-07-30 18:37:04       True
3       1       47     5.0 2000-07-30 19:03:35       True
4       1       50     5.0 2000-07-30 18:48:51       True (28934, 5)
   UserID  MovieID  Rating            Datetime  Favorable
0       1        1     4.0 2000-07-30 18:45:03       True
1       1        3     4.0 2000-07-30 18:20:47       True
2       1        6     4.0 2000-07-30 18:37:04       True
3       1       47     5.0 2000-07-30 19:03:35       True
4       1       50     5.0 2000-07-30 18:48:51       True (18398, 5)
(100836, 5)


In [29]:
favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in favorable_ratings.groupby("UserID")["MovieID"])#每个用户各喜欢哪些电影
print(favorable_reviews_by_users)

{1: frozenset({1024, 1, 1025, 3, 2048, 1029, 6, 1031, 1032, 2054, 2058, 2571, 527, 1552, 1042, 2580, 1049, 2078, 543, 3617, 1060, 1573, 2596, 552, 553, 2090, 2094, 47, 2096, 1073, 50, 1587, 2099, 3639, 1080, 2105, 2616, 1089, 1090, 2115, 1092, 2116, 2628, 1097, 3147, 590, 592, 593, 1617, 2640, 596, 1620, 2641, 2644, 2648, 1625, 2137, 2139, 3671, 2141, 2654, 2143, 608, 3168, 101, 1127, 110, 1136, 2161, 3702, 3703, 2174, 2692, 2700, 2193, 3729, 661, 151, 2716, 157, 3740, 3744, 163, 1196, 1197, 1198, 3253, 1206, 1208, 1210, 1213, 1214, 1220, 1732, 1222, 1224, 2761, 1226, 3273, 3793, 216, 1240, 2268, 733, 2273, 3809, 231, 1256, 235, 2797, 1265, 1777, 2291, 1270, 1275, 1278, 1793, 1282, 260, 2826, 1291, 1804, 1805, 1298, 2329, 804, 2858, 2353, 2872, 3386, 2366, 1348, 333, 2387, 2899, 2395, 349, 356, 2916, 2406, 362, 367, 3439, 3440, 3441, 3448, 3450, 2427, 1920, 2944, 2947, 2948, 2949, 1927, 2959, 2450, 919, 3479, 923, 2459, 3489, 1954, 2470, 4006, 2985, 2987, 940, 2478, 943, 1967, 2991, 29

In [30]:
num_favorable_by_movie = ratings[["MovieID", "Favorable"]].groupby("MovieID").sum()
num_favorable_by_movie.head()

,Favorable
MovieID,
1,49.0
2,17.0
3,11.0
4,0.0
5,10.0


In [31]:
from collections import defaultdict

def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])

In [32]:
import sys
frequent_itemsets = {}  # itemsets are sorted by length
min_support = 50

# k=1 candidates are the isbns with more than min_support favourable reviews
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"])
                                for movie_id, row in num_favorable_by_movie.iterrows()
                                if row["Favorable"] > min_support)

print("There are {} movies with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_support))
sys.stdout.flush()
for k in range(2, 20):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1],
                                                   min_support)
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        #print(cur_frequent_itemsets)
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets
# We aren't interested in the itemsets of length 1, so remove those
del frequent_itemsets[1]

There are 20 movies with more than 50 favorable reviews
I found 145 frequent itemsets of length 2
I found 508 frequent itemsets of length 3
I found 989 frequent itemsets of length 4
I found 1146 frequent itemsets of length 5
I found 811 frequent itemsets of length 6
I found 338 frequent itemsets of length 7
I found 73 frequent itemsets of length 8
I found 5 frequent itemsets of length 9
Did not find any frequent itemsets of length 10


In [33]:
print("Found a total of {0} frequent itemsets".format(sum(len(itemsets) for itemsets in frequent_itemsets.values())))

Found a total of 4015 frequent itemsets


In [35]:
#找关联规则，输出前五条看看
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))
print(candidate_rules[:5])

There are 19361 candidate rules
[(frozenset({47}), 2571), (frozenset({2571}), 47), (frozenset({47}), 527), (frozenset({527}), 47), (frozenset({47}), 50)]


In [36]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}

In [37]:
min_confidence = 0.9
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() if confidence > min_confidence}
print(len(rule_confidence))

3413


In [39]:
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")
#print(sorted_confidence)
#整体看了一下，有好多条置信度都为1的

Rule #1
Rule: If a person recommends frozenset({260, 47, 527}) they will also recommend 2571
 - Confidence: 1.000

Rule #2
Rule: If a person recommends frozenset({2858, 260, 47}) they will also recommend 2571
 - Confidence: 1.000

Rule #3
Rule: If a person recommends frozenset({356, 260, 47}) they will also recommend 2571
 - Confidence: 1.000

Rule #4
Rule: If a person recommends frozenset({1210, 50, 1196}) they will also recommend 260
 - Confidence: 1.000

Rule #5
Rule: If a person recommends frozenset({1210, 2571, 1196}) they will also recommend 260
 - Confidence: 1.000



In [66]:
test_dataset = df[~df['UserID'].isin(range(200))]
test_favorable = test_dataset[test_dataset["Favorable"]]

#test_not_favourable = test_dataset[~test_dataset["Favourable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v in test_favorable.groupby("UserID")["MovieID"])

In [67]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

In [68]:
test_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                   for candidate_rule in rule_confidence}
print(len(test_confidence))

3413


In [69]:
sorted_test_confidence = sorted(test_confidence.items(), key=itemgetter(1), reverse=True)
print(sorted_test_confidence[:5])

[((frozenset({1210, 589, 1198}), 1196), 1.0), ((frozenset({1210, 260, 589, 1198}), 1196), 1.0), ((frozenset({1210, 2571, 589, 1198}), 1196), 1.0), ((frozenset({1210, 858, 260, 589}), 1196), 1.0), ((frozenset({608, 50, 2571, 858}), 296), 1.0)]


In [70]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_test_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends frozenset({1210, 589, 1198}) they will also recommend 1196
 - Confidence: 0.944

Rule #2
Rule: If a person recommends frozenset({1210, 260, 589, 1198}) they will also recommend 1196
 - Confidence: 0.941

Rule #3
Rule: If a person recommends frozenset({1210, 2571, 589, 1198}) they will also recommend 1196
 - Confidence: 0.938

Rule #4
Rule: If a person recommends frozenset({1210, 858, 260, 589}) they will also recommend 1196
 - Confidence: 0.944

Rule #5
Rule: If a person recommends frozenset({608, 50, 2571, 858}) they will also recommend 296
 - Confidence: 0.923

